In [1]:
from rdflib import Graph, URIRef, BNode, Literal, Namespace
from rdflib.namespace import RDF, RDFS

In [2]:
objParser = {
    "authority": "Metodologia",
    "tokenLSCH": "sh85084414",
    "variant" : "Filosofia - Metodologia"
}

In [26]:
def CreateSubject(objParser):

    MADSRDF = Namespace("http://www.loc.gov/mads/rdf/v1#")

    token = objParser.get("tokenLSCH")
    authority = objParser.get("authority")
    authorityLit = Literal(authority, lang='pt')
       

    
    g = Graph()
    g.bind("madsrdf", MADSRDF)
    g.bind("rdf", RDF)

    subject = URIRef(f"https://id.loc.gov/authorities/subjects/{token}")
    g.add((subject, RDF.type, MADSRDF.Authority))
    g.add((subject, RDF.type, MADSRDF.Topic))
    g.add((subject, MADSRDF.authoritativeLabel, authorityLit))

    variantBN = BNode()
    g.add((subject, MADSRDF.hasVariant, variantBN))
    g.add((variantBN, RDF.type, MADSRDF.ComplexSubject))
    g.add((variantBN, RDF.type, MADSRDF.Variant))
    g.add((variantBN, MADSRDF.variantLabel, Literal("Philosophy--Methodology", lang='pt')))

    componentListBN = BNode()
    g.add((variantBN, MADSRDF.componentList, componentListBN))
    variantBN1 = BNode()
    g.add((componentListBN, RDF.first, variantBN1))
    g.add((variantBN1, RDF.type, MADSRDF.Topic))
    g.add((variantBN1, RDF.type, MADSRDF.Variant))
    g.add((variantBN1, MADSRDF.variantLabel, Literal("Philosophy", lang='pt')))
    firstBN = BNode()
    g.add((componentListBN, RDF.rest, firstBN))

    variantBN2 = BNode()
    g.add((firstBN, RDF.first, variantBN2))
    g.add((variantBN2, RDF.type, MADSRDF.Topic))
    g.add((variantBN2, RDF.type, MADSRDF.Variant))
    g.add((variantBN2, MADSRDF.variantLabel, Literal("Methodology", lang='pt')))
    
    g.add((firstBN, RDF.rest, RDF.nil))
    # g.add((componentListBN, RDF.first, firstBN))
    # g.add((firstBN, RDF.type, MADSRDF.Topic))
    # g.add((firstBN, RDF.type, MADSRDF.Variant))
    # g.add((firstBN, MADSRDF.variantLabel, Literal("Philosophy", lang='pt')))
    # restBN = BNode()
    # g.add((firstBN, RDF.rest, restBN))
    # g.add((restBN, RDF.type, MADSRDF.Topic))
    # g.add((restBN, RDF.type, MADSRDF.Variant))
    # g.add((restBN, MADSRDF.variantLabel, Literal("Methodology", lang='pt')))
    # g.add((firstBN, RDF.rest, RDF.nil))


    g.serialize('TESTE.nt')

CreateSubject(objParser)

